# بررسی نظرات اسنپ‌فود

در این پروژه قصد داریم داده های مربوط به نظرات و کامنت‌های رستوران‌های اسنپ‌فود را مورد بررسی قرار دهیم.
هر کامنت شامل نام و آی‌دی و سایر اطلاعات هویتی است به همراه یک متن چند جمله‌ای و یک امتیاز عددی یک در بازه‌ی یک تا ده است و ما کانت های مربوط به رستوران‌های مطرح تهران را جمع‌آوری کرده‌ایم و آن‌ها را بر اساس امتیازی که داه‌اند گرده‌بندی کرده‌ایم و کامنت های مربوط به هر امتیاز را در یک فایل مجزا قرار داده‌ایم. به طور مثال کانت های مربوط به امتیاز های ۱۰ را فایلی به نام 10.csv قرار داده‌ایم که در آدرس data_collector/comments_by_rate/10.csv قابل دستیابی است.

حال میخواهیم بررسی کنیم ببینیم لغات به کار رفته در کامنت هایی که امتیاز بالا داده‌اند از چه ترکیب و توزیعی برخوردار‌اند و همچنین کانت های مربوط به امتیاز های پایین از چه لغاتی استفاده کرده‌اند و سپس این‌ها را با هم مقایسه کنیم.

In [1]:
comments_files_path = "/home/mohammad/Desktop/Advanced Information Retrieval/snapfood/data_collector/comments_by_rate"

In [2]:
! pip install numpy
! pip install nltk
! pip install hazm
! pip install pandas
! pip install codecs
! pip install tqdm


  Using cached hazm-0.7.0-py3-none-any.whl (316 kB)
Processing /home/mohammad/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23/libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl
Processing /home/mohammad/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346/nltk-3.3-py3-none-any.whl
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
ERROR: Could not find a version that satisfies the requirement codecs (from versions: none)
ERROR: No matching distribution found for codecs


In [3]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os


In [94]:
comments_collection = {}
for i in range(11):
    comments_collection[i] = []
    file_path = comments_files_path + "/" + str(i) + ".csv"
    if os.path.isfile(file_path):
        comments_collection[i] = [x.strip().split() for x in tqdm.tqdm(codecs.open(file_path,'rU','utf-8').readlines())]

100%|███████████████████████████████████████████████████████████████████████████| 1728/1728 [00:00<00:00, 151095.67it/s]


حال داده ها را نرمال می‌کنیم:

In [98]:
from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()

comments_normalized = {}
for i in range(11):
    comments_normalized[i] = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(comments_collection[i])]

100%|█████████████████████████████████████████████████████████████████████████████| 1690/1690 [00:01<00:00, 1598.36it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 539/539 [00:00<00:00, 1236.25it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████| 764/764 [00:00<00:00, 1365.98it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1728/1728 [00:01<00:00, 1364.73it/s]


In [100]:
from nltk import FreqDist
import itertools
import pandas as pd

tokens_by_rate = {}

for i in range(11):
    sentences = [sent_tokenize(' '.join(x)) for x in tqdm.tqdm(comments_normalized[i])]
    tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(sentences)]
    tokens_by_rate[i] = list(itertools.chain(*itertools.chain(*tokens)))




100%|████████████████████████████████████████████████████████████████████████████| 1690/1690 [00:00<00:00, 26951.27it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 539/539 [00:00<00:00, 17694.58it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 764/764 [00:00<00:00, 25578.70it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████| 1728/1728 [00:00<00:00, 23788.72it/s]


حال لغات مربوط به امتیاز های خوب را از لغات مربوط به امتیاز های بد جدا می‌کنیم. 

In [101]:
bad_food_threshold = 5
good_food_threshold = 8


bad_tokens = []
for i in range(bad_food_threshold):
    bad_tokens += tokens_by_rate[i]

good_tokens = []
for i in range(good_food_threshold, 11):
    good_tokens += tokens_by_rate[i]


حال لغات پر تکرار را حذف می‌کنیم.

In [102]:
stopwords_file_path = "/home/mohammad/Desktop/Advanced Information Retrieval/snapfood/notebook/resources/stopwords.txt"
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open(stopwords_file_path,'r','utf-8').readlines()]

In [103]:
bad_tokens = [t for t in tqdm.tqdm(bad_tokens) if t not in stopwords]
good_tokens = [t for t in tqdm.tqdm(good_tokens) if t not in stopwords]



100%|███████████████████████████████████████████████████████████████████████| 140498/140498 [00:00<00:00, 546914.73it/s]


لغات مشترک را حذف می‌کنیم.

In [108]:
from collections import Counter
bad_top_tokens = [key for key, value in Counter(bad_tokens).most_common()][:100]
good_top_tokens = [key for key, value in Counter(good_tokens).most_common()][:100]

common_words = set(bad_top_tokens).intersection(good_top_tokens)

final_bad_tokens = [t for t in tqdm.tqdm(bad_tokens) if t not in common_words]
final_good_tokens = [t for t in tqdm.tqdm(good_tokens) if t not in common_words]



100%|████████████████████████████████████████████████████████████████████████| 74433/74433 [00:00<00:00, 4162761.57it/s]


حال جدول را رسم می‌کنیم:

In [109]:

dataframe = {}
dataframe['bad food'] = FreqDist(final_bad_tokens).most_common(50)
dataframe['good food'] = FreqDist(final_good_tokens).most_common(50)

freq_analysis = pd.DataFrame(dataframe) 

freq_analysis

,bad food,good food
0,"(پایین, 436)","(موقع, 452)"
1,"(افتضاح, 361)","(داغ, 404)"
2,"(مونده, 357)","(خوش, 402)"
3,"(ساعت, 319)","(سریع, 324)"
4,"(خمیر, 235)","(ممنونم, 270)"
5,"(میداد, 229)","(چی, 247)"
6,"(داخل, 223)","(یکم, 237)"
7,"(نوشابه, 218)","(بودم, 223)"
8,"(خشک, 217)","(بودعالی, 217)"
9,"(بوی, 215)","(عالیه, 188)"
